# Who are our top performers? Find matching locations
---
### Summary
    1. Import Modules
    2. Request JSON Through Census API, Convert To Pandas DF
    3. Clean Data
        A. Drop Useless Columns
        B. Rename Columns
        C. Datatypes Format
        D. Null and Negative Values
    4. Data Exploration
        A. Import CSV With Zip Frequency And Monetary Values
        B. What Are The Demographics Of Our Top Zips?
    5. Compares Differences
        A. Create Differences Function
        B. Pivot Data 
        C. Findings
    6. Save Data

## 1. Import Modules

In [171]:
import pandas as pd
import requests

# See all data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

## 2. Request JSON Through Census API, Convert To Pandas DF

In [102]:
# Demographics we will use:
# -total population B01001_001E 
# -median age B01002_001E 
# -home age median B25035_001E 
# -median home value B25107_001E 
# -median household income last 12 mo B19013_001E 
# -home owner lived in unit B07013_002E

api_variables = "B01001_001E,B01002_001E,B25035_001E,B25107_001E,B19013_001E,B07013_002E"
url = "https://api.census.gov/data/2019/acs/acs5?get={}&for=zip%20code%20tabulation%20area:*&in=state:08"\
                                                                                    .format(api_variables)

payload={}
headers = {
  'Cookie': 'TS010383f0=011ba694f273cea421f27bf0ab2cac56fcf022cae435637d6ff0\
                      4b48f22db0da4930c5fdccf2e3ecfe07edeb712ddebd5998d53170'
}

response = requests.request("GET", url, headers=headers, data=payload)


In [103]:
def json_to_dataframe(response):
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [104]:
df = json_to_dataframe(response)

df.head(2)

,B01001_001E,B01002_001E,B25035_001E,B25107_001E,B19013_001E,B07013_002E,state,zip code tabulation area
0,42,59.0,0,-666666666,27000,42,08,80434
1,4257,48.7,1995,276800,68272,2864,08,80446


## 4. Clean Data

### A. Drop Useless Columns

In [105]:
df = df.drop(columns=["state"])

df.head(1)

,B01001_001E,B01002_001E,B25035_001E,B25107_001E,B19013_001E,B07013_002E,zip code tabulation area
0,42,59.0,0,-666666666,27000,42,80434


### B. Rename Columns

In [106]:
column_update = ["population", #B01001_001E
                 "median_age", #B01002_001E
                 "median_home_age", #B25035_001E
                 "median_home_value", #B25107_001E
                 "median_income", #B19013_001E
                 "owner_occ", # B07013_002E
                 "zip"]

df.columns = column_update

In [107]:
df.head()

,population,median_age,median_home_age,median_home_value,median_income,owner_occ,zip
0,42,59.0,0,-666666666,27000,42,80434
1,4257,48.7,1995,276800,68272,2864,80446
2,2328,48.3,1983,381300,74809,1431,80447
3,15858,34.5,1970,203500,57234,9818,80701
4,7180,37.0,1972,171800,46897,4784,80723


### C. Datatypes Format

In [108]:
df.dtypes

population           object
median_age           object
median_home_age      object
median_home_value    object
median_income        object
owner_occ            object
zip                  object
dtype: object

In [109]:
df = df.apply(pd.to_numeric, errors='coerce')

In [110]:
df.dtypes

population           int64  
median_age           float64
median_home_age      int64  
median_home_value    int64  
median_income        int64  
owner_occ            int64  
zip                  int64  
dtype: object

### D. Null Or Negative Values

In [111]:
# Check for null values
df.isnull().sum()

population           0
median_age           0
median_home_age      0
median_home_value    0
median_income        0
owner_occ            0
zip                  0
dtype: int64

In [112]:
# Check for negative numbers
# Census data that is negative signifies no data collected
df.lt(0).sum()

population           0 
median_age           18
median_home_age      20
median_home_value    54
median_income        47
owner_occ            0 
zip                  0 
dtype: int64

In [114]:
# Check to see if zips with negative values are usefull
df[(df < 0).values].head()

,population,median_age,median_home_age,median_home_value,median_income,owner_occ,zip
0,42,59.0,0,-666666666,27000,42,80434
12,3728,21.7,2006,-666666666,54732,0,80913
18,0,-666666666.0,1987,-666666666,-666666666,0,80448
18,0,-666666666.0,1987,-666666666,-666666666,0,80448
18,0,-666666666.0,1987,-666666666,-666666666,0,80448


In [115]:
# We are going to drop all rows with negative values because they all have a very low population, are a military base,
# or have multiple negative values making them useless
df = df[df > 0].dropna()
df.lt(0).sum()

population           0
median_age           0
median_home_age      0
median_home_value    0
median_income        0
owner_occ            0
zip                  0
dtype: int64

## 5. Data Exploration

### A. Import CSV With Zip Frequency And Monetary Values

In [116]:
df_f_m = pd.read_csv("zip_f_m.csv")

### B. What Are The Demographics Of Our Top Zips?

In [117]:
# Top zips by total monetary value spent
df_f_m[df_f_m['label'] == 'Top'].sort_values('monetary_sum', ascending=False).head(5)

,location_zip,frequency_count,monetary_sum,f_score,m_score,score_total,label
0,80919,1653,489861.74,1,1,2,Top
1,80920,1641,466744.75,1,1,2,Top
4,80906,1236,449015.50,1,1,2,Top
2,80921,1309,347282.33,1,1,2,Top
3,80132,1268,311673.35,1,1,2,Top


In [118]:
# Top zips by total appointments 
df_f_m[df_f_m['label'] == 'Top'].sort_values('frequency_count', ascending=False).head(5)

,location_zip,frequency_count,monetary_sum,f_score,m_score,score_total,label
0,80919,1653,489861.74,1,1,2,Top
1,80920,1641,466744.75,1,1,2,Top
2,80921,1309,347282.33,1,1,2,Top
3,80132,1268,311673.35,1,1,2,Top
4,80906,1236,449015.50,1,1,2,Top


In [119]:
# Create array of selected zips
df_list = df_f_m[df_f_m['label'] == 'Top'].sort_values('monetary_sum', ascending=False).head(5)
zip_array = df_list['location_zip'].tolist()
zip_array

[80919, 80920, 80906, 80921, 80132]

In [120]:
# Demographics of top zips
top_zip_demos = df[df['zip'].isin(zip_array)]
top_zip_demos

,population,median_age,median_home_age,median_home_value,median_income,owner_occ,zip
33,40016.0,35.6,1992.0,332000.0,96284.0,28689.0,80920
89,28039.0,43.6,1988.0,373500.0,95320.0,21143.0,80919
311,37608.0,39.1,1984.0,357800.0,68701.0,23312.0,80906
312,24087.0,41.5,2004.0,444900.0,124085.0,19779.0,80921
389,21286.0,43.9,1996.0,474200.0,129009.0,19009.0,80132


## 6. Compares Differences

### A. Create Differences Function

In [121]:
def create_df(target_zip):
    """
    Loop through all CO zips finding the 
    differences between target zip
    demographics
    """
    arr = [] 
    for index, row in df.iterrows(): 
        for column in df.columns:   
            if column != 'zip':
                target_zip_data = df[df['zip'] == target_zip]['{}'.format(column)].iat[0]
                other_zip_data = row['{}'.format(column)]

                case = {"selected_zip": int(target_zip), "comparing_zip": int(row['zip']), "category": column, "selected_zip_data": target_zip_data, 
                                "comparing_zip_data": other_zip_data, "difference": abs(target_zip_data-other_zip_data),
                                "difference_percent": abs(target_zip_data-other_zip_data) / ((target_zip_data + other_zip_data) / 2) * 100}
                arr.append(case)
    created_df = pd.DataFrame(arr)
    return created_df

In [164]:
df_80920 = create_df(80920)
df_80906 = create_df(80906)
df_80921 = create_df(80921)
df_80132 = create_df(80132)
df_80919 = create_df(80919)
df_80919.head(6)

,selected_zip,comparing_zip,category,selected_zip_data,comparing_zip_data,difference,difference_percent
0,80919,80446,population,28039.0,4257.0,23782.0,147.275204
1,80919,80446,median_age,43.6,48.7,5.1,11.050921
2,80919,80446,median_home_age,1988.0,1995.0,7.0,0.351494
3,80919,80446,median_home_value,373500.0,276800.0,96700.0,29.740120
4,80919,80446,median_income,95320.0,68272.0,27048.0,33.067632
5,80919,80446,owner_occ,21143.0,2864.0,18279.0,152.280585


### B. Pivot Data

In [135]:
def create_piv(df, zip_string):
    """
    Pivot the df, aggregate all values,
    top 10 lowest values sorted saved 
    as CSV
    """
    piv = df.pivot(index='comparing_zip',
              columns='category',
              values='difference_percent')
    piv['mean_diffs']= piv.mean(axis=1)           
    piv['selected_zip'] = zip_string
    piv = piv.sort_values('mean_diffs').head(11) 
    return piv

In [136]:
# Concat all dfs to final pivot df
final_piv = pd.concat([create_piv(df_80920, "80920"), create_piv(df_80906, "80906"), 
                      create_piv(df_80921, "80921"), create_piv(df_80132, "80132"),
                      create_piv(df_80919, "80919")])

In [137]:
# Final data
final_piv = final_piv.reset_index()
final_piv.head()

category,comparing_zip,median_age,median_home_age,median_home_value,median_income,owner_occ,population,mean_diffs,selected_zip
0,80920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80920
1,80241,3.042877,0.100351,2.174885,6.129243,10.468784,17.318815,6.539159,80920
2,80128,14.099217,0.503271,13.036745,1.289891,3.345101,9.368092,6.940386,80920
3,80601,6.376812,0.350789,5.668267,23.969667,7.504114,1.856849,7.621083,80920
4,80031,6.784261,0.352024,4.964018,15.696004,6.898044,11.286086,7.663406,80920


### C. Findings

In [167]:
# Zip code mode, sorted
zip_modes = final_piv['comparing_zip'].mode().tolist()
final_piv[(final_piv['comparing_zip'].isin(zip_modes))&(~final_piv['comparing_zip'].isin(zip_array))].sort_values('mean_diffs')

category,comparing_zip,median_age,median_home_age,median_home_value,median_income,owner_occ,population,mean_diffs,selected_zip
23,80109,13.642214,0.049888,1.028623,1.067727,0.511950,1.767465,3.011311,80921
14,80003,5.519054,0.454660,11.528229,3.837852,6.795956,3.045028,5.196796,80906
35,80023,5.620609,0.499750,21.005945,1.113839,1.633143,6.248720,6.020334,80132
25,80023,0.000000,0.099751,27.290373,5.004341,2.337536,6.109614,6.806936,80921
36,80403,5.140187,0.301054,12.011893,12.323796,7.459551,3.766902,6.833897,80132
2,80128,14.099217,0.503271,13.036745,1.289891,3.345101,9.368092,6.940386,80920
37,80109,19.225968,0.449888,5.348057,2.823611,3.458526,10.584859,6.981818,80132
16,80031,2.590674,0.050391,2.518749,17.976843,13.831656,5.090731,7.009841,80906
26,80130,10.126582,0.400000,2.288635,1.498936,12.752059,18.481885,7.591350,80921
4,80031,6.784261,0.352024,4.964018,15.696004,6.898044,11.286086,7.663406,80920


## 7. Save Data

In [141]:
# Save final csv files
df.to_csv("all_zip_demos.csv", encoding='utf-8', index=True)
top_zip_demos.to_csv("top_zip_demos.csv", encoding='utf-8', index=True)
final_piv.to_csv("final_piv.csv", encoding='utf-8', index=True)